<a href="https://colab.research.google.com/github/David9857/VQA/blob/main/KAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import spacy
import spacy.cli
spacy.cli.download("en_vectors_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_vectors_web_lg')


In [ ]:
!mkdir ./results

In [ ]:
!python /content/drive/MyDrive/KAN/train.py

Traceback (most recent call last):
  File "/content/drive/MyDrive/KAN/train.py", line 2, in <module>
    from core.exec import Execution
  File "/content/drive/.shortcut-targets-by-id/1qq5LCjZ3AGmL2Kv20y4s0nEVp9K7G-Mn/KAN/core/exec.py", line 2, in <module>
    from core.model.net import Net
ModuleNotFoundError: No module named 'core.model.net'


In [ ]:
!mkdir ./data
!cp /content/drive/MyDrive/data_path.zip ./data/
!cd ./data && unzip data_path.zip

In [ ]:
!cd ./data && unzip data_path.zip

In [ ]:
# !ls /content/data/data/ques_embeddings/bluebert
!cp -r /content/data/data/ques_embeddings/w2v /content/drive/MyDrive/VQA_path/ques_embeddings

In [ ]:
!ls /content/drive/MyDrive/VQA_path/ques_embeddings/bioelmo/7563.npy

/content/drive/MyDrive/VQA_path/ques_embeddings/bioelmo/7563.npy


In [ ]:
!ls /content/drive/MyDrive/VQA_path/ques_embeddings/bioelmo | wc -l

32761


In [ ]:
!ls /content/data/data/ques_embeddings/bioelmo/7563.npy

/content/data/data/ques_embeddings/bioelmo/7563.npy


In [ ]:
!unzip -uq '/content/drive/MyDrive/data_path.zip' -d '/content/drive/MyDrive/data_path'

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import pandas as pd
import time
import pathlib
from utils.load_data import DataLoader
from utils.evaluation import AnswerEvaluator
from utils.training_toolkit import CustomSchedule, loss_function
from models.Transformer.transformers import VQATransformer
from models.Transformer.masks import create_masks

In [ ]:
class Image_Question_Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, maximum_position_encoding,
                 rate=0.1 ,pretrained_cnn_type='inception', multiplier = 10):
        super(Image_Question_Encoder, self).__init__()
        self.maximum_position_encoding = maximum_position_encoding
        self.pretrained_CNN = pretrained_cnn(pretrained_cnn_type)
        self.fc1 = Dense(maximum_position_encoding * multiplier) # fully connected layer
        self.image_dense = Dense(d_model, activation='relu')
        self.question_encoder = Encoder(
            num_layers=num_layers,
            d_model=d_model,
            num_heads=num_heads,
            dff=dff,
            maximum_position_encoding=maximum_position_encoding)
        self.concatenator = Concatenate(axis=1)
        self.conv = tf.keras.layers.Conv2D(1024, 3, padding='same', activation='relu')
        self.bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True))

    def call(self, question, img, training, mask):
        img_feature = self.pretrained_CNN(img)  # (7, 7, 1024)
        img_feature = self.conv(img_feature)
        img_shape = img_feature.shape[-1]
        img_feature = tf.keras.layers.Reshape((-1, img_shape))(img_feature)  # (49, 1024)
        img_feature = self.fc1(img_feature)  # (49, pe_input)
        img_feature = tf.transpose(img_feature, perm=(0, 2, 1))  # (Res, 49)
        img_feature = tf.keras.layers.Reshape((self.maximum_position_encoding, -1))(img_feature)
        img_feature = self.image_dense(img_feature)  # (pe_input, 512)
        question = self.bilstm(question)
        context = self.question_encoder(question, img_feature, training, mask)  # (batch, question_len, 512)
        # context = self.concatenator([img_feature, question_feature]) # （batch, 64 + question_len, 512)
        return context  # (64, 108, 512)

In [ ]:
transformer = VQATransformer(num_layers, d_model, num_heads, dff, vocab_size, pe_input, pe_output,
                          pretrained_cnn_type=cnn_type)

learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, check_point_path, max_to_keep=5)

# Download and unzip data

In [ ]:
!cp /content/drive/MyDrive/files/data_path.zip /content
!unzip data_path.zip "data/jsons/*" "data/pic/*" "data/ques_embeddings/bioelmo/*" "data/answer_word_frequency.csv" "data/answer_word_frequency.xlsx" -d /content

# Object detection

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
# test installation
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
!cp /content/models/research/object_detection/colab_tutorials/inference_from_saved_model_tf2_colab.ipynb /content/drive/MyDrive/Colab\ Notebooks

# Entity extraction

In [ ]:
# The textual entities are KG
# entities associated with key phrases of the question, where we utilize the Stanford NLP Dependency Parser [4] and Stanford Named
# Entity Recognizer [5] to analyze the question and extract all noun
# phrases, verb-object phrases and named entities as the key phrases.


In [6]:
from sklearn.utils import shuffle
import pandas as pd
import pathlib

root = pathlib.Path('/content/data')
d = shuffle(pd.read_json(root.joinpath('jsons/open_ended').with_suffix('.json')), random_state=1)

In [16]:
dq = d['Questions']
print(dq)

12134                                     What is present?
9969                                      What is present?
3785                                     What is present ?
2947                            What does this image show?
858                     What are colors of these hormones?
                               ...                        
7813                                      What is present?
10955                           What does this image show?
5192                            Where does this belong to?
12172                                     What is present?
235      How many copies does the red probe hybridize t...
Name: Questions, Length: 16429, dtype: object


In [24]:
print(dq[0])

What are positively charged,  thus allowing the compaction of the negatively charged DNA?


In [21]:
pip install stanfordcorenlp

In [55]:
import spacy
nlp = spacy.load("en_core_web_sm")
# text = ("When Sebastian Thrun started working on self-driving cars at "
#         "Google in 2007, few people outside of the company took him "
#         "seriously. “I can tell you very senior CEOs of major American "
#         "car companies would shake my hand and turn away because I wasn’t "
#         "worth talking to,” said Thrun, in an interview with Recode earlier "
#         "this week.")
def ectract_query(ques):
  qs = nlp(ques)
  queries = []
  for tk in qs:
    # print(tk.text,'|',tk.pos_)
    if tk.pos_ == 'VERB' or tk.pos_ == 'NOUN':
      queries.append(tk.lemma_)
  return queries


print(ectract_query(dq[0]))

['charge', 'allow', 'compaction', 'charge', 'dna']


In [69]:
queries = ectract_query(dq[0])
for q in queries:
  kns = query_knowledge(q)
  print(kns)

# query_knowledge

['charge related to credit', 'charge related to card', 'human at location charge', 'accuse synonym charge', 'bill synonym charge']
['allow related to permit', 'allow related to let', 'allow related to permission', 'empower synonym allow', 'permit synonym allow']
['densification synonym compaction', 'compression synonym compaction', 'concretion synonym compaction', 'compaction is a concentration', 'compaction synonym crush']
['charge related to credit', 'charge related to card', 'human at location charge', 'accuse synonym charge', 'bill synonym charge']
['dna is a molecule', 'dna at location cell', 'dna has property short for deoxyribonucleic acid', 'dna is a complex molecule', 'dna is a double stranded molecule']


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import json
import re
import os

tokenizer = AutoTokenizer.from_pretrained("bert-large-cased", do_lower_case=True)
model = AutoModel.from_pretrained("bert-large-cased", output_hidden_states=True)

with open('../data/jsons/QA.json', 'r', encoding='utf-8') as f:
    data = json.load(f)


def preprocess_sentence(w):
    w = w.lower().strip()
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z ]", "", w)
    w = w.strip()
    return w


path = '../data/ques_embeddings/bert'
if not os.path.exists(path):
    os.mkdir(path)

for qa in data:
    ques_id = qa['Question_Id']
    ques = qa['Questions']
    after = preprocess_sentence(ques)

    marked_text1 = "[CLS] " + after + " [SEP]"
    tokenized_text1 = tokenizer.tokenize(marked_text1)
    indexed_tokens1 = tokenizer.convert_tokens_to_ids(tokenized_text1)
    segments_ids1 = [1] * len(tokenized_text1)
    tokens_tensor1 = torch.tensor([indexed_tokens1])
    segments_tensors1 = torch.tensor([segments_ids1])

    with torch.no_grad():
        # Because we set 'output_hidden_states = True', the third item will be the
        # hidden states from all layers.
        outputs1 = model(tokens_tensor1, segments_tensors1)
        hidden_states1 = outputs1[2]

    # Concatenate the tensors for all layers.
    # Use 'stack' to create a new dimension in the tensor.
    token_embeddings1 = torch.stack(hidden_states1, dim=0)
    # Remove dimension 1, the "batches".
    token_embeddings1 = torch.squeeze(token_embeddings1, dim=1)
    # Swap dimensions 0 and 1.（“layers” and “tokens” dimensions）
    token_embeddings1 = token_embeddings1.permute(1, 0, 2)

    token_vecs_sum1 = []

    for token in token_embeddings1:
        # Sum the vectors from the last four layers.
        sum_vec1 = torch.sum(token[-4:], dim=0)
        token_vecs_sum1.append(sum_vec1)
    token_vec = np.array([t.numpy() for t in token_vecs_sum1])
    # Write .npy file to '/content/embedding/' folder.
    np.save(os.path.join(path, str(ques_id)+'.npy'), token_vec)
    print("saved file: ", ques_id)
print("saved all")

# validate saved features and calculate max length of all questions
max_len = 0
for pair in data:
    ques_id = pair['Question_Id']
    emb = np.load('../data/ques_embeddings/bert/'+str(ques_id)+'.npy')
    length = emb.shape[0]
    if length >= max_len:
        max_len = length
        print('current max length is', max_len)

In [ ]:
class DataLoader:
    def __init__(self, location, emb_folder, image_size=None):
        self.root = pathlib.Path(location)
        if not self.root.exists():  # check if path of data exist
            raise ValueError('The location of data does not exist: {}.'.format(location))

        # specify image and question feature location and size
        if image_size is None:
            image_size = [224, 224]
        self.image_size = image_size
        pic = self.root.joinpath('pic')  # get picture location
        ques = self.root.joinpath('ques_embeddings/' + emb_folder)

        # create data frame for yes_no and open-ended questions
        self.dfs = dict()
        self.dfs['open_ended'] = shuffle(pd.read_json(self.root.joinpath('jsons/open_ended').with_suffix('.json')),
                                         random_state=1)
        self.dfs['yes_no'] = shuffle(pd.read_json(self.root.joinpath('jsons/yes_no').with_suffix('.json')),
                                     random_state=1)

        # create dataframe of paths for image and question features
        for key in self.dfs.keys():
            df = self.dfs[key]
            df['image'] = df['Images'].map(lambda file: str(pic.joinpath(str(file))) + '.jpg')
            df['question'] = df['Question_Id'].map(lambda file: str(ques.joinpath(str(file))) + '.npy')

    # function for loading images
    def load_and_preprocess_image(self, path):
        image = tf.io.read_file(path) # read image file
        image = tf.image.decode_jpeg(image, channels=3) # decode image
        image = tf.image.resize(image, self.image_size)
        return image

    # function for loading question embeddings
    @staticmethod
    def load_question_features(path):
        return np.load(path)

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:-
    # https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    @staticmethod
    def process_answer(w):
        w = w.lower().strip()
        w = re.sub(r"([?.!,¿])", r" \1 ", w)
        w = re.sub(r'[" "]+', " ", w)
        # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
        w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
        w = w.strip()
        # adding a start and an end token to the sentence
        # so that the model know when to start and stop predicting.
        w = '<start> ' + w + ' <end>'
        return w

    # function for creating dataset
    def create_dataset(self, ques_type='yes_no'):
        assert ques_type in ['yes_no', 'open_ended'], "Invalid question type."

        # get question id for datasets, useful for result analysis
        ques_id_ds = tf.data.Dataset.from_tensor_slices(self.dfs[ques_type]['Question_Id'])

        # create image dataset
        img_path_ds = tf.data.Dataset.from_tensor_slices(self.dfs[ques_type]['image'])
        image_ds = img_path_ds.map(self.load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

        # create question feature dataset
        ques_path_ds = tf.data.Dataset.from_tensor_slices(self.dfs[ques_type]['question'])
        ques_ds = ques_path_ds.map(lambda x: tf.numpy_function(self.load_question_features, inp=[x], Tout=tf.float32),
                                   num_parallel_calls=tf.data.experimental.AUTOTUNE)

        # create answer dataset
        answers = self.dfs[ques_type]['Answers'].map(lambda x: self.process_answer(x))
        # use tokenizer for string and one-hot mapping, counting vocab, max length
        tokenizer = tf.keras.preprocessing.text.Tokenizer(filters="", oov_token="<unk>", lower=True)
        tokenizer.fit_on_texts(answers)
        answers = tokenizer.texts_to_sequences(answers)
        # use 0 as padding
        tokenizer.word_index['<pad>'] = 0
        tokenizer.index_word[0] = '<pad>'
        answers = tf.keras.preprocessing.sequence.pad_sequences(answers, padding='post')
        ans_ds = tf.data.Dataset.from_tensor_slices(answers)

        return tf.data.Dataset.zip(((image_ds, ques_ds), ans_ds, ques_id_ds)), tokenizer

# Knowledge

## SenticNet

In [ ]:
pip install senticnet

     |████████████████████████████████| 51.9MB 142kB/s 


In [ ]:
from senticnet.senticnet import SenticNet

sn = SenticNet()
concept_info = sn.concept('love')
polarity_label = sn.polarity_label('love')
polarity_value = sn.polarity_value('love')
moodtags = sn.moodtags('love')
semantics = sn.semantics('love')
sentics = sn.sentics('love')

In [ ]:
print(concept_info)
print(polarity_label)
print(polarity_value)
print(moodtags)
print(semantics)
print(sentics)

{'polarity_label': 'positive', 'polarity_value': '0.83', 'sentics': {'introspection': '0.897', 'temper': '0', 'attitude': '0', 'sensitivity': '0.762'}, 'moodtags': ['#joy', '#eagerness'], 'semantics': ['desire_succeed', 'excitement', 'lust', 'passion', 'sexuality']}
positive
0.83
['#joy', '#eagerness']
['desire_succeed', 'excitement', 'lust', 'passion', 'sexuality']
{'introspection': '0.897', 'temper': '0', 'attitude': '0', 'sensitivity': '0.762'}


## ConceptNet

In [32]:
pip install conceptnet-lite

     |████████████████████████████████| 51kB 2.4MB/s 
     |████████████████████████████████| 2.3MB 6.0MB/s 
     |████████████████████████████████| 870kB 7.9MB/s 
  Created wheel for peewee: filename=peewee-3.14.4-cp37-cp37m-linux_x86_64.whl size=592991 sha256=841f97788286615ee578f0a91f5f311e873dee1bd6e668cffcafdf647ebd2ea5
  Stored in directory: /root/.cache/pip/wheels/a0/ce/ff/732bd2a2729233b3a55bb5d8557d500f7be41bd21df3f01048
  Created wheel for lmdb: filename=lmdb-0.97-cp37-cp37m-linux_x86_64.whl size=219753 sha256=e2320c125ee7e5af52d5a2810d61f459a0bd6936c99c9c577a770a9fbbc4faa9
  Stored in directory: /root/.cache/pip/wheels/7f/4d/44/b299a9b22d8eaaaa455304687fb6d4c02d9178c1530850e12a
Successfully built peewee lmdb
  Found existing installation: lmdb 0.99
    Uninstalling lmdb-0.99:
      Successfully uninstalled lmdb-0.99


In [34]:
import conceptnet_lite

conceptnet_lite.connect("/path/to/conceptnet.db")

File not found: /path/to/conceptnet.db
Download compressed database
[*] 0 bytes / 1.85 GB @ 0 bytes/s [------------------] [0.0%, 0s left]    [*] 2.0 MB / 1.85 GB @ 10.0 MB/s [------------------] [0.1%, 0s left]    [*] 9.4 MB / 1.85 GB @ 31.4 MB/s [------------------] [0.5%, 0s left]    [*] 18.4 MB / 1.85 GB @ 46.0 MB/s [------------------] [1.0%, 0s left]    [*] 26.2 MB / 1.85 GB @ 52.4 MB/s [------------------] [1.4%, 0s left]    [*] 33.4 MB / 1.85 GB @ 55.7 MB/s [------------------] [1.8%, 0s left]    [*] 43.7 MB / 1.85 GB @ 62.5 MB/s [-------------

In [ ]:
from conceptnet_lite import Label

cat_concepts = Label.get(text='charge', language='en').concepts  
for c in cat_concepts:
    print("    Concept URI:", c.uri)
    print("    Concept text:", c.text)
    print("    Concept language:", c.language.name)

In [68]:
from conceptnet_lite import Label, edges_for

def query_knowledge(q):
  d = {}
  for e in edges_for(Label.get(text=q, language='en').concepts, same_language=True):
    # print(e.start.text, "::", e.end.text, "|", e.relation.name, "|", e.etc['weight'])
    d[(e.start.text,e.relation.name,e.end.text)] = e.etc['weight']
    # print(e.etc['weight'])
  kn = []
  for i,f in enumerate(sorted(d.items(), key=lambda item: item[1], reverse=True)):
    if i == 5:
      break
    k = ' '.join(f[0])
    k = k.replace('_', ' ')
    kn.append(k)
  return kn

print(query_knowledge('dna'))

['dna is a molecule', 'dna at location cell', 'dna has property short for deoxyribonucleic acid', 'dna is a complex molecule', 'dna is a double stranded molecule']


In [56]:
for i,f in enumerate(sorted(d.items(), key=lambda item: item[1], reverse=True)):
  if i == 5:
    break
  print(f)
  print(f[0])